In [4]:
import pandas as pd
import numpy as np
import re, string, nltk

In [5]:
my_file_path = "C:\Project\\twtr\data\Tweets.csv" # Change this to your file path

In [6]:
df = pd.read_csv(my_file_path)
df.shape

(14640, 15)

In [7]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [8]:
def remove_mentioned_pattern(text, pattern = "@[\w]*"):
    return re.sub(pattern, '', text)

# df['processed_text'] = df['text'].apply(remove_mentioned_pattern)
df['processed_text'] = np.vectorize(remove_mentioned_pattern)(df['text'])


In [9]:
def remove_special_characters(text, pattern="[^a-zA-Z# ]"):
   return re.sub(pattern, '', text)

df['processed_text'] = np.vectorize(remove_special_characters)(df['processed_text'])

In [10]:
def remove_short_words(text, min_length=3):
    return " ".join([word for word in text.split() if len(word)>=min_length])

df['processed_text'] = np.vectorize(remove_short_words)(df['processed_text'])

In [11]:
try:
    nltk.corpus.stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
finally:
    from nltk.corpus import stopwords
    sw = stopwords.words('english')

In [12]:
def remove_stopwords(text):
    return " ".join([word.lower() for word in text.split() if word.lower() not in sw])

df['processed_text'] = np.vectorize(remove_stopwords)(df['processed_text'])

In [14]:
try:
    nltk.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
finally:
    from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_words(input):
    return [lemmatizer.lemmatize(word) for word in input.split()]

df['lemmatized_text'] = df['processed_text'].apply(lemmatize_words)

In [15]:
df[['text', 'processed_text', 'lemmatized_text']].head()

,text,processed_text,lemmatized_text
0,@VirginAmerica What @dhepburn said.,said,[said]
1,@VirginAmerica plus you've added commercials t...,plus youve added commercials experience tacky,"[plus, youve, added, commercial, experience, t..."
2,@VirginAmerica I didn't today... Must mean I n...,didnt today must mean need take another trip,"[didnt, today, must, mean, need, take, another..."
3,@VirginAmerica it's really aggressive to blast...,really aggressive blast obnoxious entertainmen...,"[really, aggressive, blast, obnoxious, enterta..."
4,@VirginAmerica and it's a really big bad thing...,really big bad thing,"[really, big, bad, thing]"
